In [1]:
import tqdm
import os
import re
import codecs
import jieba
import pickle
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch.client import IndicesClient

In [2]:
es = Elasticsearch([{'host':'localhost','port':9200}])

In [3]:
es_idx = IndicesClient(es)

In [144]:
a = os.popen('''
curl -X PUT 'localhost:9200/main_idx' -d '
{
  "mappings": {
    "docs": {
      "properties": {
        "pure_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        },
        "processed_text": {
          "type": "text",
          "analyzer": "ik_max_word",
          "search_analyzer": "ik_max_word"
        }
      }
    }
  }
}'
''').read()
for line in a.split('\n'):
    print(line)

{"acknowledged":true,"shards_acknowledged":true,"index":"main_idx"}


In [35]:
# res1 = os.popen('''
# curl -X PUT 'localhost:9200/main_idx/docs/1' -d '
# {
#     "pure_text": "中华人民共和国国歌",
#     "processed_text": "中华 人民 共和国 国歌"
# }'
# ''').read()
# for line in res1.split('\n'):
#     print(line)

{"_index":"main_idx","_type":"docs","_id":"1","_version":1,"result":"created","_shards":{"total":2,"successful":1,"failed":0},"created":true}


In [39]:
# res = os.popen('''
# curl 'localhost:9200/main_idx/docs/_search'  -d '
# {
#   "query" : { "match" : { "pure_text" : "中华人民共和国" }}
# }'
# ''').read()
# for line in res.split('\n'):
#     print(line)

{"took":1,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":1,"max_score":0.25811607,"hits":[{"_index":"main_idx","_type":"docs","_id":"1","_score":0.25811607,"_source":
{
    "pure_text": "中华人民共和国国歌",
    "processed_text": "中华 人民 共和国 国歌"
}}]}}


In [70]:
# es_idx.exists(index='text_search')

False

In [143]:
es_idx.delete(index='_all')

{'acknowledged': True}

In [149]:
# all_question = pd.read_csv('./data/description.csv')
Herceptin_question = pd.read_excel('./data/Herceptin.xlsx')
# all_question = pd.read_excel('./data/Herceptin_funding_filtered.xlsx')
# all_question = pd.read_excel('./data/Herceptin_patient_filtered.xlsx')
Kadcyla_question = pd.read_excel('./data/Kadcyla.xlsx')
Perjeta_1_question = pd.read_excel('./data/Perjeta_1.xlsx')
Perjeta_2_question = pd.read_excel('./data/Perjeta_2.xlsx')
Tecentriq_question = pd.read_excel('./data/Tecentriq.xlsx')
Xeloda_question = pd.read_excel('./data/Xeloda.xlsx')

all_question = pd.concat([Herceptin_question, Kadcyla_question, Perjeta_1_question, Perjeta_2_question, Tecentriq_question, Xeloda_question])
print('all_question: ', len(all_question))
all_question.head(5)

all_question.dropna(axis=0, inplace=True, subset=['Description'])
all_question.isna().sum()

print('all_question: ', len(all_question))

jieba.load_userdict('./dict data/chinese_word_dict.txt')
chinese_word_dict = pickle.load(open('./dict data/chinese_word_dict.pickle', 'rb'))
stopwords = [line.strip() for line in codecs.open('./stopwords-zh.txt', 'r', encoding='utf8').readlines()]

def keyword_extract(text, word_dict):
#     text = list(jieba.cut(text, cut_all=False))
    text = list(jieba.cut_for_search(text))
    union_text = list(set(text).intersection(set(word_dict)))
    return " ".join(union_text)

all_question['KeyWords'] = all_question.Description.apply(lambda x: keyword_extract(x, chinese_word_dict))

# 去除所有“[]”内的文本
def remove_special_string(text):
    text = re.sub(r'\[[^]]*\]', '', text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r'（[^）]*\）', '', text)
    text = re.sub(r'\<[^>]*\>', '', text)
    text = re.sub(r'[0-9]*', '', text)
#     text = re.sub('患者', '', text)
#     text = re.sub('家属', '', text)
#     text = re.sub('咨询', '', text)
#     text = re.sub('询问', '', text)
    return text
def remove_stopwords(text, stopwords):
    text_cutted = jieba.cut(text, cut_all=True)
    text_removed = []
    for word in text_cutted:
        if word in stopwords or word.strip()=="":
            continue
        text_removed.append(word)
    return "  ".join(text_removed)

all_question['Description'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Description.apply(lambda x: remove_special_string(x))
all_question['Processed'] = all_question.Processed.apply(lambda x: remove_stopwords(x, stopwords))
all_question.head()

all_question:  1407
all_question:  1093


,Case Number,Description,KeyWords,Processed
0,1420656.0,赫赛汀 配了之后结冰了， 能否继续使用。,赫赛汀 使用,赫赛汀 配 之后 结冰 继续 使用
1,1423956.0,赫赛汀粉末形状如何？患者保存的赫赛汀的稀释液出现结冰情况，是否能否使用,无 赫赛汀 使用 患者,赫赛汀 粉末 形状 保存 赫赛汀 稀释 稀释液 出现 结冰 冰情 情况 ...
2,1434184.0,请帮忙查一些赫赛汀与血栓栓塞方面的数据和文献,查 血栓 无 赫赛汀 栓塞 血栓栓塞 文献,帮忙 查 赫赛汀 血栓 血栓栓塞 栓塞 方面 数据 和文 文献
3,1436011.0,拿到的赫赛汀呈现块状是否正常,赫赛汀 正常,拿到 赫赛汀 呈现 块状 是否 正常
4,1438071.0,常规配置和保存的赫赛汀复溶后的溶液保存在-度的情况下，现出现浑浊情况，能否使用,下 使用 现 赫赛汀 配置 溶液,常规 配置 保存 赫赛汀 复 溶 溶液 保存 存在 度 情况 现出 ...


In [166]:
for idx, row in all_question.iterrows():
    if '二聚体' in row['Description']:
        print(row['Description'])

 赫赛汀抑制二聚体形成的临床研究


In [165]:
doc_list = []
for idx,row in all_question.iterrows():
    doc = {"pure_text": row['Description'], "processed_text": row['KeyWords']}
    doc_list.append(doc)
print(len(doc_list))
print(doc_list[0])

1093
{'pure_text': '赫赛汀  配了之后结冰了， 能否继续使用。', 'processed_text': '赫赛汀 使用'}


In [159]:
res1 = os.popen('''
curl -X PUT 'localhost:9200/main_idx/docs/1' -d '
{
    "pure_text": "中华人民共和国国歌",
    "processed_text": "中华 人民 共和国 国歌"
}'
''').read()
for line in res1.split('\n'):
    print(line)

{"_index":"ik_test","_type":"docs","_id":"1","_version":3,"result":"updated","_shards":{"total":2,"successful":1,"failed":0},"created":false}


In [ ]:
for doc in doc_list:
    temp = os.popen('''
    curl -X POST 'localhost:9200/main_idx/docs' -d '
    {}
    '
    '''.format(str(doc).replace("\"", "\'").replace("\'", "\""))).read()
    for line in temp.split('\n'):
        print(line)
        
#     print('''
#     curl -X PUT 'localhost:9200/ik_test/docs/{}' -d '
#     {}
#     '
#     '''.format(str(idx), str(doc_list[idx]).replace("\"", "\'").replace("\'", "\"")))
    
# curl -X PUT 'localhost:9200/ik_test/docs/1' -d '
# {
#     "content": "中华人民共和国国歌"
# }'

In [ ]:
res = os.popen('''
curl 'localhost:9200/main_idx/docs/_search?pretty=true'  -d '
{
  "query" : { "match" : { "processed_text" : "关于赫赛汀心脏毒性：1、临床 2、临床前" }}
}'
''').read()
for line in res.split('\n'):
    print(line)

In [96]:
for idx in range(len(doc_list)):
    es.index(index='text_search', id=idx, body=doc_list[idx])

In [168]:

query = "赫赛汀抑制二聚体形成的临床研究"

# processed_text  pure_text
results = es.search(index='main_idx', body=
# {
#   "query" : { "match" : { "pure_text" : query }}
# }
{
  "query": {
    "bool": {
      "must":     { "match": { "processed_text": query }},
      "should":   { "match": { "pure_text": "赫赛汀" }}
    }
  }
}
,size=20)
hits = results['hits']['hits']
# results
for idx, res in enumerate(hits):
    print(str(idx+1) + ":  " + res['_source']['pure_text'])

1:   赫赛汀抑制二聚体形成的临床研究
2:   麻烦查询“BOLERO-”临床研究的全文，如果没有全文，请尽量提供更多的该研究的信息，谢谢
3:  ，帕捷特什么时候上市，价格是多少，有无临床研究。
4:   需要罗氏帕妥珠单抗晚期一线，辅助，新辅助临床研究发表的原文。
5:   乳腺癌晚期一线TX对比TE的临床研究
6:  你好：请在clinicaltrial.gov网站上查询是否已经有这样的临床设计注册：关于HER阳性新辅助未达pCR，然后分组一组H，一组HX。H：赫赛汀；HX：赫赛汀联合希罗达。想自行发起这样的临床研究，需要先查询是否已经有类似的临床已经注册，谢谢！
7:   请在clinicaltrial.gov网站上查询是否已经有这样的临床设计注册：关于HER阳性新辅助未达pCR，然后分组一组H，一组HX。H：赫赛汀；HX：赫赛汀联合希罗达。医生想自行发起这样的临床研究，想请我们先查询是否已经有类似的临床已经注册，谢谢！
8:   卡培联合AI，从体外实验到临床，研究文献
9:   您好，请帮忙查询晚期乳腺癌一线治疗使用赫赛汀联合卡培他滨的临床研究，全文，谢谢
10:  卡培他滨研发早期用于晚期乳腺癌临床研究文献
11:   请问现有有关帕捷特的临床研究中有紫杉醇三周方案的吗？如果有，请提供研究以紫杉的具体用法。如果没有，麻烦解释原因，谢谢！
12:   需要一些转移性乳腺癌，xt-x化疗的相关的临床研究和资料。谢谢。
13:  ，帕托珠单抗在晚期乳腺癌的相关临床研究
14:   晚期乳腺癌XT方案和GP方案对比的临床研究，XT方案与其它联合方案的临床研究
15:   请查询帕妥珠单抗用于早期乳腺癌新辅助，辅助以及晚期乳腺癌的临床研究原文文献。
16:   希罗达在晚期乳腺癌使用的文献及临床研究，年的NCCN乳腺癌诊疗指南，年ASCO乳腺癌诊疗指南，能否麻烦把手头上有的发给我呢？谢谢
17:   请帮忙在https://clinicaltrials.gov/网站进行查询，是否有氟纬司琼联合卡培他滨节拍化疗应用在晚期乳腺癌的临床研究注册，如果有，请提供注册的详细内容，谢谢
18:   麻烦帮我查询早期乳腺癌新辅助治疗中含有卡培他滨的所有化疗方案的临床研究原文，谢谢
19:   三阴性乳腺癌用卡培辅助一年的临床研究有哪个？是否有具体的文献，谢谢
20:   

In [65]:
1:  关于赫赛汀心脏毒性：1、临床 2、临床前
2:  [Wechat] 临床上赫赛汀q3W和qW用法，在心脏毒性方面，哪个频率心脏毒性小些？相关研究？
3:  [Wechat] 关于tdm1的临床试验
4:  [Wechat] 请帮忙查询以下临床研究的全文以供学习，谢谢！ 1、XT对比GT在晚期乳腺癌化疗中的临床研究原文；重点关注毒性的对比。至少有两个相关研究。X：希罗达，T：多西他赛/多西紫杉醇，G：吉西他滨  2、GP（吉西他滨联合紫杉醇）用于晚期乳腺癌化疗的临床研究全文。
5:  [Wechat] 赫赛汀对于心脏毒性的相关文献以及处理方法
6:  [Wechat] 赫赛汀，放疗，心脏毒性相关文献
7:  [Wechat] 关于拉帕替尼加希罗达的临床试验
8:  [Wechat] 你好，我要问询关于帕捷特使用的问题，目前临床有一位乳腺癌患者体重30kg，帕捷特应该怎么使用，临床老师担心，针对这种特殊患者是否需要减量，比如6周一个疗程
9:  [Wechat] 都是关于希罗达在晚期乳腺癌的研究：1.多西他赛联合希罗达对比多西他赛单药，的3期临床研究原文（中文）2.希罗达 多西他赛vs 表柔比星 多西他赛的临床研究原文（中文）3.希罗达 多西他赛 vs 吉西他滨 多西他赛的临床研究原文（中文）谢谢您
10:  赫赛汀在心脏毒性方面的研究数据

SyntaxError: invalid character in identifier (<ipython-input-65-f0049b57d55b>, line 1)

In [167]:
tokens_res = es_idx.analyze(index='main_idx', body={
    "analyzer": "ik_max_word",
#     "analyzer": "ik_smart",
    "text" : "赫赛汀抑制二聚体形成的临床研究"
})
for token in tokens_res['tokens']:
    print(token['token'])

赫赛汀
抑制
二聚体
聚
体形
体
形成
形
成
临床研究
临床
床
研究
究
